
# DMI (+DI/−DI) + ADX Momentum **Screener** (NSE)

Filter a user-provided list of NSE stocks for **long/short** swing setups using **+DI/−DI** with an **ADX** trend-strength filter per the uploaded PDF (enter only when ADX shows a strong trend, default ≈ 25). fileciteturn0file0

**Outputs**: CSV with `Ticker, Date, Signal, Close, +DI, -DI, ADX` saved under `outputs/` with a date tag (today or your **AS-OF** date).  
**Notes**: Uses `pandas_ta` (no TA-Lib), yfinance, IST timezone normalization, no plotting.



## 0) Environment (install if needed)
Uncomment and run once if these packages are missing:
```python
# %pip install yfinance pandas pandas_ta numpy pytz
```


In [3]:

import os, warnings, math
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from zoneinfo import ZoneInfo

warnings.filterwarnings("ignore")
pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 200)

# ==============================
# USER PARAMETERS (EDIT THESE)
# ==============================
# 1) Universe (replace with your list; keep Yahoo '.NS' suffix)
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GNFC.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GPPL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JUSTDIAL.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KNRCON.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PEL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RAYMOND.NS', 'RAYMONDLSL.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RENUKA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'ROUTE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RTNINDIA.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SWSOLAR.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TANLA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']



# 2) Data settings
INTERVAL = "1d"          # daily bars
LOOKBACK_YEARS = 5

# 3) DMI/ADX parameters
DMI_LEN = 14
ADX_THRESHOLD = 25.0     # strong trend filter

# 4) Signal recency window
# 0  => require signal on the last bar available (as-of date)
# N>0 => accept any signal within the last N bars (as-of aware)
RECENT_BARS = 0

# 5) AS-OF date (IST). Use 'YYYY-MM-DD'. Set to "" to use latest available.
ASOF_DATE_STR = "2025-04-21"       # e.g., "2024-12-31"

# 6) Output directory
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

IST = ZoneInfo("Asia/Kolkata")


def _compute_date_bounds(asof_str: str, years: int):
    # Return (start_date_str, end_date_str or None, asof_dt) in IST.
    if asof_str:
        asof_dt = datetime.strptime(asof_str, "%Y-%m-%d").replace(tzinfo=IST)
        start = (asof_dt - timedelta(days=365*years)).date().isoformat()
        end = (asof_dt + timedelta(days=1)).date().isoformat()  # yfinance end is exclusive
        return start, end, asof_dt
    else:
        now = datetime.now(tz=IST)
        start = (now - timedelta(days=365*years)).date().isoformat()
        return start, None, None


def fetch_ohlcv_asof(ticker: str, years: int, interval: str, asof_str: str) -> pd.DataFrame:
    start, end, asof_dt = _compute_date_bounds(asof_str, years)
    df = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        return df
    # Localize to IST
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert(IST)
    else:
        df.index = df.index.tz_convert(IST)
    df = df.rename(columns=str.title)
    if asof_dt is not None:
        df = df[df.index <= asof_dt]
    return df


def compute_dmi(df: pd.DataFrame, length: int) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    adx = ta.adx(out["High"], out["Low"], out["Close"], length=length)
    if adx is None or adx.empty:
        out["+DI"] = np.nan
        out["-DI"] = np.nan
        out["ADX"] = np.nan
        return out
    out["+DI"] = adx[f"DMP_{length}"]
    out["-DI"] = adx[f"DMN_{length}"]
    out["ADX"] = adx[f"ADX_{length}"]
    return out


def generate_signals(df: pd.DataFrame, adx_th: float) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out["bull_cross"] = (out["+DI"] > out["-DI"]) & (out["+DI"].shift(1) <= out["-DI"].shift(1))
    out["bear_cross"] = (out["-DI"] > out["+DI"]) & (out["-DI"].shift(1) <= out["+DI"].shift(1))
    strong = out["ADX"] >= adx_th
    out["LongSignal"]  = out["bull_cross"] & strong
    out["ShortSignal"] = out["bear_cross"] & strong
    return out


# ==============================
# Screener
# ==============================
rows = []

for tk in TICKERS:
    try:
        df = fetch_ohlcv_asof(tk, LOOKBACK_YEARS, INTERVAL, ASOF_DATE_STR)
        if df.empty or len(df) < DMI_LEN + 5:
            continue

        df = compute_dmi(df, DMI_LEN)
        df = generate_signals(df, ADX_THRESHOLD).dropna(subset=["+DI","-DI","ADX"])

        if df.empty:
            continue

        if RECENT_BARS <= 0:
            last = df.iloc[-1]
            if last["LongSignal"] or last["ShortSignal"]:
                rows.append({
                    "Ticker": tk,
                    "Date": df.index[-1].date().isoformat(),
                    "Signal": "LONG" if last["LongSignal"] else "SHORT",
                    "Close": round(float(last["Close"]), 2),
                    "+DI": round(float(last["+DI"]), 2),
                    "-DI": round(float(last["-DI"]), 2),
                    "ADX": round(float(last["ADX"]), 2),
                })
        else:
            window = df.iloc[-RECENT_BARS:]
            hits = window[(window["LongSignal"]) | (window["ShortSignal"])]
            for idx, row in hits.iterrows():
                rows.append({
                    "Ticker": tk,
                    "Date": idx.date().isoformat(),
                    "Signal": "LONG" if row["LongSignal"] else "SHORT",
                    "Close": round(float(row["Close"]), 2),
                    "+DI": round(float(row["+DI"]), 2),
                    "-DI": round(float(row["-DI"]), 2),
                    "ADX": round(float(row["ADX"]), 2),
                })
    except Exception as e:
        print(f"[WARN] {tk}: {e}")

res_df = pd.DataFrame(rows).sort_values(["Signal","ADX"], ascending=[True, False]).reset_index(drop=True)
display(res_df)

# ==============================
# Save CSV
# ==============================
if ASOF_DATE_STR:
    tag_date = ASOF_DATE_STR
else:
    tag_date = datetime.now(tz=IST).date().isoformat()

out_path = os.path.join(OUT_DIR, f"dmi_adx_candidates_{tag_date}.csv")
if res_df.empty:
    print(f"No signals found for configured parameters as of {tag_date}.")
else:
    res_df.to_csv(out_path, index=False)
    print(f"Saved {len(res_df)} candidates -> {out_path}")



1 Failed download:
['COHANCE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-04-22 -> 2025-04-22) (Yahoo error = "Data doesn\'t exist for startDate = 1587493800, endDate = 1745260200")')


KeyError: 'Signal'